# This Notebook contains code for web Scraping. 

It is Scraping Daraz Product details for the first 5 pages. It can used for extrating product details from any market place by adjusting parameters according to web page HTML structure

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

def extract_product_info():
    title_element = None
    price_element = None
    delivery_element = None
    sellername_element = None
    sellerrating_element = None
    shipontime_element = None
    daraz_status = None
    rating_element= None

    try:
        title_element = driver.find_element(By.CSS_SELECTOR, '#module_product_title_1 > div > div > span').text
    except NoSuchElementException:
        print("Title not found")


        
    try:
        price_element = driver.find_element(By.CLASS_NAME, 'pdp-price').text.strip()
    except NoSuchElementException:
        print("Price not found")

    try:
        delivery_element = driver.find_element(By.CSS_SELECTOR, '#module_seller_delivery .delivery-option-item__title span:nth-child(1)').text.strip()
    except NoSuchElementException:
        print("Delivery option not found")

    try:
        sellername_element = driver.find_element(By.CLASS_NAME, 'pdp-link_theme_black.seller-name__detail-name').text.strip()
    except NoSuchElementException:
        print("Seller name not found")

    try:
        sellerrating_element = driver.find_element(By.CLASS_NAME, 'seller-info-value.rating-positive').text
    except NoSuchElementException:
        print("Seller rating not found")

    try:
        shipontime_element = driver.find_element(By.CLASS_NAME, 'seller-info-value').text
    except NoSuchElementException:
        print("Shipping on time information not found")
    
    try:
        daraz_element = driver.find_element(By.CLASS_NAME,'pdp-seller-badge')
        daraz_status = True
    except NoSuchElementException:
        print("Not on Daraz Mall")
        daraz_status = False
    
    try:
        driver.execute_script("window.scrollTo(0, 850);")
        time.sleep(1)
        rating_element=driver.find_element(By.CLASS_NAME,'score').text
    except NoSuchElementException:
        print("No rating for this product")

    return {
        "Title": title_element,
        "Price": price_element,
        "Delivery": delivery_element,
        "SellerName": sellername_element,
        "SellerRating": sellerrating_element,
        "ShipOnTime": shipontime_element,
        "Daraz Mall Status": daraz_status,
        "Ratings": rating_element,
    }


driver.get('https://www.daraz.pk/catalog/?_keyori=ss&clickTrackInfo=textId--8106960466929852021__abId--None__pvid--51001bb4-fe5c-406a-98ef-7f1bf391c08e__matchType--1__abGroup--None__srcQuery--mobile%20phones__spellQuery--mobile%20phones__ntType--nt-common&from=suggest_normal&page=1&q=mobile%20phones&spm=a2a0e.home.search.1.6a274076lTRNcE&sugg=mobile%20phones_0_1')

columns = ["Title", "Price", "Delivery", "SellerName", "SellerRating", "ShipOnTime", "Daraz Mall Status", "Ratings"]
df = pd.DataFrame(columns=columns)
num_pages = 6
products_per_page = 40

for page in range(num_pages):
    print(f"Extracting data from page {page + 1}...")
    
    product_elements = driver.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')

    products_extracted = 0

    for index, product_element in enumerate(product_elements):
        try:
            print(f"Extracting data for product {index + 1} on page {page + 1}...")
            
            # Check if the 'a' tag is present within the product_element
            try:
                product_url_element = product_element.find_element(By.TAG_NAME, 'a')
                if product_url_element:
                    product_url = product_url_element.get_attribute('href')

                    # Open the product URL in a new tab or window
                    driver.execute_script("window.open('');")
                    driver.switch_to.window(driver.window_handles[1])
                    driver.get(product_url)

                    try:
                        product_info = extract_product_info()

                        if product_info:
                            row_data = pd.Series(product_info)
                            df = df.append(row_data, ignore_index=True)
                            products_extracted += 1
                    except NoSuchElementException:
                        print("Error extracting product info. Skipping.")

                    # Close the tab or window and switch back to the main page
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

            except StaleElementReferenceException:
                print("Stale element reference. Skipping.")
                continue  # Skip to the next product if there's a stale element reference

        except NoSuchElementException:
            print("Product URL not found. Skipping.")

        if products_extracted >= products_per_page:
            break

    if page < num_pages:
        try:
            driver.execute_script("window.scrollTo(0, 3850);")
            time.sleep(10)
            page_number = page + 1
            print(f"Clicking on page {page_number} button...")
            page_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, f'ant-pagination-item-{page_number}'))
            )
            page_button.click()
            time.sleep(2)  
        except NoSuchElementException:
            print(f"Page button for page {page_number} not found. Exiting.")
            break
        except Exception as e:
            print(f"An error occurred: {e}")
            break

# Close the browser
driver.quit()

print(df)

Extracting data from page 1...
Extracting data for product 1 on page 1...


C:\Users\HP\AppData\Local\Temp\ipykernel_17204\4055675547.py:128: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)


Extracting data for product 2 on page 1...


C:\Users\HP\AppData\Local\Temp\ipykernel_17204\4055675547.py:128: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)


Extracting data for product 3 on page 1...
Seller rating not found
Not on Daraz Mall


...

Not on Daraz Mall


C:\Users\HP\AppData\Local\Temp\ipykernel_17204\4055675547.py:128: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)


Extracting data for product 40 on page 6...
Not on Daraz Mall


C:\Users\HP\AppData\Local\Temp\ipykernel_17204\4055675547.py:128: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)


Clicking on page 6 button...
                                                 Title        Price  \
0    Redmi Note 12 (8GB-128GB) PTA Approved With Of...   Rs. 51,999   
1    Like New Phones - Used Apple iPhone X - Silver...  Rs. 106,999   
2    Realme Narzo 50A Prime (4GB-128GB) PTA Approve...   Rs. 32,999   
3    Infinix Note 30 Pro 8-256 GB PTA Approved With...   Rs. 64,499   
4    Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...   Rs. 64,999   
..                                                 ...          ...   
235  Samsung A24 - 6GB RAM 128GB ROM - 6.6 Inches -...   Rs. 70,999   
236  Infinix Smart 7 , 4GB RAM 64GB ROM , Octa Core...   Rs. 25,999   
237  Samsung A34 5G - 8GB RAM 128GB/256GB ROM - 6.6...  Rs. 115,999   
238  Vivo V27 e RAM 8 GB ROM 256 GB Front Camera 32...  Rs. 119,999   
239  Xiaomi Redmi 12 RAM 8 GB ROM 128 GB Front Came...   Rs. 45,000   

              Delivery                    SellerName SellerRating  ShipOnTime  \
0        Free Delivery               

In [11]:
df

,Title,Price,Delivery,SellerName,SellerRating,ShipOnTime,Daraz Mall Status,Ratings
0,Redmi Note 12 (8GB-128GB) PTA Approved With Of...,"Rs. 51,999",Free Delivery,Xiaomi,93%,93%,True,4.7
1,Like New Phones - Used Apple iPhone X - Silver...,"Rs. 106,999",Standard Delivery,Daraz Like New,92%,92%,True,5.0
2,Realme Narzo 50A Prime (4GB-128GB) PTA Approve...,"Rs. 32,999",Free Delivery,Teletel Store,None,New Seller,False,4.1
3,Infinix Note 30 Pro 8-256 GB PTA Approved With...,"Rs. 64,499",Free Delivery,Teletel Store,None,New Seller,False,4.7
4,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,"Rs. 64,999",Standard Delivery,Daraz,92%,92%,True,4.7
...,...,...,...,...,...,...,...,...
235,Samsung A24 - 6GB RAM 128GB ROM - 6.6 Inches -...,"Rs. 70,999",Free Delivery,Kashif Electronics (Karachi),89%,89%,False,None
236,"Infinix Smart 7 , 4GB RAM 64GB ROM , Octa Core...","Rs. 25,999",Free Delivery,Bint e Saeed,87%,87%,False,5.0
237,Samsung A34 5G - 8GB RAM 128GB/256GB ROM - 6.6...,"Rs. 115,999",Free Delivery,Mercato trading,91%,91%,False,None
238,Vivo V27 e RAM 8 GB ROM 256 GB Front Camera 32...,"Rs. 119,999",Free Delivery,Ali Communication (Khanewal),95%,95%,False,5.0


In [12]:
df.to_csv('details.csv',index=False)

In [13]:
df.drop_duplicates(inplace=True)
print(df)

                                                 Title        Price  \
0    Redmi Note 12 (8GB-128GB) PTA Approved With Of...   Rs. 51,999   
1    Like New Phones - Used Apple iPhone X - Silver...  Rs. 106,999   
2    Realme Narzo 50A Prime (4GB-128GB) PTA Approve...   Rs. 32,999   
3    Infinix Note 30 Pro 8-256 GB PTA Approved With...   Rs. 64,499   
4    Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...   Rs. 64,999   
..                                                 ...          ...   
235  Samsung A24 - 6GB RAM 128GB ROM - 6.6 Inches -...   Rs. 70,999   
236  Infinix Smart 7 , 4GB RAM 64GB ROM , Octa Core...   Rs. 25,999   
237  Samsung A34 5G - 8GB RAM 128GB/256GB ROM - 6.6...  Rs. 115,999   
238  Vivo V27 e RAM 8 GB ROM 256 GB Front Camera 32...  Rs. 119,999   
239  Xiaomi Redmi 12 RAM 8 GB ROM 128 GB Front Came...   Rs. 45,000   

              Delivery                    SellerName SellerRating  ShipOnTime  \
0        Free Delivery                        Xiaomi          93% 

In [14]:
df

,Title,Price,Delivery,SellerName,SellerRating,ShipOnTime,Daraz Mall Status,Ratings
0,Redmi Note 12 (8GB-128GB) PTA Approved With Of...,"Rs. 51,999",Free Delivery,Xiaomi,93%,93%,True,4.7
1,Like New Phones - Used Apple iPhone X - Silver...,"Rs. 106,999",Standard Delivery,Daraz Like New,92%,92%,True,5.0
2,Realme Narzo 50A Prime (4GB-128GB) PTA Approve...,"Rs. 32,999",Free Delivery,Teletel Store,None,New Seller,False,4.1
3,Infinix Note 30 Pro 8-256 GB PTA Approved With...,"Rs. 64,499",Free Delivery,Teletel Store,None,New Seller,False,4.7
4,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,"Rs. 64,999",Standard Delivery,Daraz,92%,92%,True,4.7
...,...,...,...,...,...,...,...,...
235,Samsung A24 - 6GB RAM 128GB ROM - 6.6 Inches -...,"Rs. 70,999",Free Delivery,Kashif Electronics (Karachi),89%,89%,False,None
236,"Infinix Smart 7 , 4GB RAM 64GB ROM , Octa Core...","Rs. 25,999",Free Delivery,Bint e Saeed,87%,87%,False,5.0
237,Samsung A34 5G - 8GB RAM 128GB/256GB ROM - 6.6...,"Rs. 115,999",Free Delivery,Mercato trading,91%,91%,False,None
238,Vivo V27 e RAM 8 GB ROM 256 GB Front Camera 32...,"Rs. 119,999",Free Delivery,Ali Communication (Khanewal),95%,95%,False,5.0


In [15]:
df.to_csv("details.csv",index=False)

In [3]:
import pandas as pd
df=pd.read_csv("details.csv")
keywords_to_exclude = ['case', 'cover', 'charger', 'cable']
df_filtered = df[~df['Title'].str.lower().str.contains('|'.join(keywords_to_exclude).lower(), na=False)].copy()
df_filtered


,Title,Price,Delivery,SellerName,SellerRating,ShipOnTime,Daraz Mall Status,Ratings
0,Redmi Note 12 (8GB-128GB) PTA Approved With Of...,"Rs. 51,999",Free Delivery,Xiaomi,93%,93%,True,4.7
1,Like New Phones - Used Apple iPhone X - Silver...,"Rs. 106,999",Standard Delivery,Daraz Like New,92%,92%,True,5.0
2,Realme Narzo 50A Prime (4GB-128GB) PTA Approve...,"Rs. 32,999",Free Delivery,Teletel Store,NaN,New Seller,False,4.1
3,Infinix Note 30 Pro 8-256 GB PTA Approved With...,"Rs. 64,499",Free Delivery,Teletel Store,NaN,New Seller,False,4.7
4,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,"Rs. 64,999",Standard Delivery,Daraz,92%,92%,True,4.7
...,...,...,...,...,...,...,...,...
194,Samsung A24 - 6GB RAM 128GB ROM - 6.6 Inches -...,"Rs. 70,999",Free Delivery,Kashif Electronics (Karachi),89%,89%,False,NaN
195,"Infinix Smart 7 , 4GB RAM 64GB ROM , Octa Core...","Rs. 25,999",Free Delivery,Bint e Saeed,87%,87%,False,5.0
196,Samsung A34 5G - 8GB RAM 128GB/256GB ROM - 6.6...,"Rs. 115,999",Free Delivery,Mercato trading,91%,91%,False,NaN
197,Vivo V27 e RAM 8 GB ROM 256 GB Front Camera 32...,"Rs. 119,999",Free Delivery,Ali Communication (Khanewal),95%,95%,False,5.0


In [4]:
df_filtered.to_csv('details_filtered.csv',index=False)

In [5]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193 entries, 0 to 198
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              191 non-null    object 
 1   Price              192 non-null    object 
 2   Delivery           191 non-null    object 
 3   SellerName         192 non-null    object 
 4   SellerRating       187 non-null    object 
 5   ShipOnTime         192 non-null    object 
 6   Daraz Mall Status  193 non-null    bool   
 7   Ratings            156 non-null    float64
dtypes: bool(1), float64(1), object(6)
memory usage: 12.3+ KB


In [4]:
import pandas as pd
df=pd.read_csv("details_filtered.csv")
df["Price"].max()

267000.0